In [1]:
# zenodo_upload_tool.py, a Python script for uploading resources through the Zenodo API.

# (c) 2024 Vanderbilt University. This program is released under a GNU General Public License v3.0 http://www.gnu.org/licenses/gpl-3.0
# Author: Steve Baskauf

version = '0.0.1'
created = '2024-04-17'

# Zenodo API developer guide: https://developers.zenodo.org/#quickstart-upload

# -----------------------------------------
# Import modules.
# -----------------------------------------

import requests
from typing import List, Dict, Tuple, Optional, Any
from pathlib import Path
import json
from time import sleep

# -----------------------------------------
# Global variables.
# -----------------------------------------

# NOTE: The access tokens for the sandbox and production Zenodo instances are different. To use the sandbox, you need
# to log in to the sandbox instance and create a new access token.

#BASE_URL = 'https://zenodo.org/api'
BASE_URL = 'https://sandbox.zenodo.org/api' # for testing
HTTP_HEADER = {
    'Content-Type': 'application/json'
    }
#API_ACCESS_TOKEN_FILENAME = 'zenodo_bioimages_upload_access_token.txt'
API_ACCESS_TOKEN_FILENAME = 'zenodo_sandbox_access_token.txt'

Load the access token

In [2]:
# Read the API access token from a file in the home directory. The file should contain only the key and no other text.
home = str(Path.home()) # gets path to home directory; supposed to work for both Win and Mac
with open(home + '/' + API_ACCESS_TOKEN_FILENAME, 'r') as file:
    api_access_token = file.read().strip() # remove any leading or trailing white space or newlines
request_params = {'access_token': api_access_token}


Test code to see if the access token is working.

In [3]:
request_url = BASE_URL + '/deposit/depositions'
r = requests.get(request_url, params=request_params)
print(r.status_code)
print(json.dumps(r.json(), indent = 2))

200
[
  {
    "created": "2024-04-17T20:18:00.576346+00:00",
    "modified": "2024-04-17T20:18:00.631979+00:00",
    "id": 46387,
    "conceptrecid": "46386",
    "metadata": {
      "access_right": "open",
      "prereserve_doi": {
        "doi": "10.5281/zenodo.46387",
        "recid": 46387
      }
    },
    "title": "",
    "links": {
      "self": "https://sandbox.zenodo.org/api/records/46387",
      "html": "https://sandbox.zenodo.org/records/46387",
      "badge": "https://sandbox.zenodo.org/badge/doi/.svg",
      "files": "https://sandbox.zenodo.org/api/records/46387/files",
      "latest_draft": "https://sandbox.zenodo.org/api/deposit/depositions/46387",
      "latest_draft_html": "https://sandbox.zenodo.org/deposit/46387",
      "publish": "https://sandbox.zenodo.org/api/deposit/depositions/46387/actions/publish",
      "edit": "https://sandbox.zenodo.org/api/deposit/depositions/46387/actions/edit",
      "discard": "https://sandbox.zenodo.org/api/deposit/depositions/46387/a

## Create an empty upload

Response format is

```
{
  "created": "2024-04-17T18:37:17.668653+00:00",
  "modified": "2024-04-17T18:37:17.721187+00:00",
  "id": 46377,
  "conceptrecid": "46376",
  "metadata": {
    "access_right": "open",
    "prereserve_doi": {
      "doi": "10.5281/zenodo.46377",
      "recid": 46377
    }
  },
  "title": "",
  "links": {
    "self": "https://sandbox.zenodo.org/api/deposit/depositions/46377",
    "html": "https://sandbox.zenodo.org/deposit/46377",
    "badge": "https://sandbox.zenodo.org/badge/doi/.svg",
    "files": "https://sandbox.zenodo.org/api/deposit/depositions/46377/files",
    "bucket": "https://sandbox.zenodo.org/api/files/a500714d-8562-45b3-9c56-b38c1d04c6b2",
    "latest_draft": "https://sandbox.zenodo.org/api/deposit/depositions/46377",
    "latest_draft_html": "https://sandbox.zenodo.org/deposit/46377",
    "publish": "https://sandbox.zenodo.org/api/deposit/depositions/46377/actions/publish",
    "edit": "https://sandbox.zenodo.org/api/deposit/depositions/46377/actions/edit",
    "discard": "https://sandbox.zenodo.org/api/deposit/depositions/46377/actions/discard",
    "newversion": "https://sandbox.zenodo.org/api/deposit/depositions/46377/actions/newversion",
    "registerconceptdoi": "https://sandbox.zenodo.org/api/deposit/depositions/46377/actions/registerconceptdoi"
  },
  "record_id": 46377,
  "owner": 9120,
  "files": [],
  "state": "unsubmitted",
  "submitted": false
}
```

In [4]:
request_url = BASE_URL + '/deposit/depositions'
payload = {}
response = requests.post(request_url, params=request_params, json=payload, headers=HTTP_HEADER)

print(response.status_code)
response_data = response.json()
print(json.dumps(response_data, indent = 2))

# Get the bucket URL from the response data
bucket_url = response_data['links']['bucket']
deposition_id = response_data['id']


201
{
  "created": "2024-04-17T21:01:13.824695+00:00",
  "modified": "2024-04-17T21:01:13.883064+00:00",
  "id": 46404,
  "conceptrecid": "46403",
  "metadata": {
    "access_right": "open",
    "prereserve_doi": {
      "doi": "10.5281/zenodo.46404",
      "recid": 46404
    }
  },
  "title": "",
  "links": {
    "self": "https://sandbox.zenodo.org/api/deposit/depositions/46404",
    "html": "https://sandbox.zenodo.org/deposit/46404",
    "badge": "https://sandbox.zenodo.org/badge/doi/.svg",
    "files": "https://sandbox.zenodo.org/api/deposit/depositions/46404/files",
    "bucket": "https://sandbox.zenodo.org/api/files/5b76780d-4b2d-4dd7-addc-bf5fb0c85069",
    "latest_draft": "https://sandbox.zenodo.org/api/deposit/depositions/46404",
    "latest_draft_html": "https://sandbox.zenodo.org/deposit/46404",
    "publish": "https://sandbox.zenodo.org/api/deposit/depositions/46404/actions/publish",
    "edit": "https://sandbox.zenodo.org/api/deposit/depositions/46404/actions/edit",
    "di

Do a PUT request to the `files` link to upload a file.

Response is:

```
{
    'created': '2024-04-17T18:50:13.617887+00:00',
    'updated': '2024-04-17T18:50:14.297914+00:00',
    'version_id': '08b0eff7-c895-4ced-b0d3-f6860cae3833',
    'key': 'lisq--flbracts36266.jpg',
    'size': 612698,
    'mimetype': 'image/jpeg',
    'checksum': 'md5:07f590493c5aff505c886b2524126b0a',
    'is_head': True,
    'delete_marker': False,
    'links': 
        {
        'self': 'https://sandbox.zenodo.org/api/files/39bcbfaf-c874-48ea-b011-c01782a093f2/lisq--flbracts36266.jpg',
        'version': 'https://sandbox.zenodo.org/api/files/39bcbfaf-c874-48ea-b011-c01782a093f2/lisq--flbracts36266.jpg?versionId=08b0eff7-c895-4ced-b0d3-f6860cae3833',
        'uploads': 'https://sandbox.zenodo.org/api/files/39bcbfaf-c874-48ea-b011-c01782a093f2/lisq--flbracts36266.jpg?uploads'}
        }
  ```

In [5]:
filename = 'casc5-fl32936.jpg'
home_subpath = '/bioimages-highres/baskauf/'
local_file_path = home + home_subpath + filename
bucket_file_url = bucket_url + '/' + filename

''' 
The target URL is a combination of the bucket link with the desired filename
seperated by a slash.
'''
with open(local_file_path, "rb") as fp:
    r = requests.put(
        bucket_file_url,
        data=fp,
        params=request_params,
    )
print(r.status_code)
print(json.dumps(r.json(), indent = 2))

201
{
  "created": "2024-04-17T21:01:21.219379+00:00",
  "updated": "2024-04-17T21:01:21.894406+00:00",
  "version_id": "73f4c367-27d9-4d0e-974f-1d6df814527c",
  "key": "casc5-fl32936.jpg",
  "size": 537811,
  "mimetype": "image/jpeg",
  "checksum": "md5:5843653a6867f6bac8c3cdaf40dd6a58",
  "is_head": true,
  "delete_marker": false,
  "links": {
    "self": "https://sandbox.zenodo.org/api/files/5b76780d-4b2d-4dd7-addc-bf5fb0c85069/casc5-fl32936.jpg",
    "version": "https://sandbox.zenodo.org/api/files/5b76780d-4b2d-4dd7-addc-bf5fb0c85069/casc5-fl32936.jpg?versionId=73f4c367-27d9-4d0e-974f-1d6df814527c",
    "uploads": "https://sandbox.zenodo.org/api/files/5b76780d-4b2d-4dd7-addc-bf5fb0c85069/casc5-fl32936.jpg?uploads"
  }
}


## Add metadata

In [40]:
deposition_id = 46381
deposition_url = request_url + '/' + str(deposition_id)
data = {
    'metadata': {
        'title': 'Liatris squarrosa (L.) Michx. - flower - side',
        'upload_type': 'poster',
        'description': 'Live plant image of a Liatris squarrosa (L.) Michx. (scaly blazing star) flower ',
        'creators': [
            {'name': 'Baskauf, Steven J.', 'affiliation': 'Vanderbilt University'}
            ]
     }
 }
r = requests.put(deposition_url, params={'access_token': api_access_token}, data=json.dumps(data),headers=HTTP_HEADER)
r.status_code

200

In [42]:
print(deposition_url)

https://sandbox.zenodo.org/api/deposit/depositions/46381


## Publish the upload

In [43]:
publication_url = deposition_url + '/actions/publish'
r = requests.post(publication_url, params=request_params)
r.status_code


202